In [ ]:
from google.colab import drive
import os

# set path to project folder
gdrive_path='/content/gdrive/MyDrive/7-programming/music_recognition/'

# mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

# navigate to Google Drive folder
os.chdir(gdrive_path)

# check that we are in the right folder
print(sorted(os.listdir()))

: 

In [ ]:
import torch
import torch.utils.data as data

class MusicDataset(data.Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        self.data = []
        for root, dirs, files in os.walk(data_path):
            for file in files:
                print(file)
                if file.endswith(".pt"):
                    self.data.append(os.path.join(root, file))

    def __getitem__(self, index):
        data = torch.load(self.data[index])
        if self.transform:
            data = self.transform(data)
        return data

    def __len__(self):
        return len(self.data)

In [ ]:
data_root = os.path.join(gdrive_path, 'data')

train_data = MusicDataset(data_path=data_root)
#val_data = MusicDataset(data_path='data/val')
#test_data = MusicDataset(data_path='data/test')